In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.special import iv

In [2]:
print('Lassi')

Lassi


![FORMULA](formula.png) 

In [3]:
# Quantities declaration
c=300000000
carrier_frequency=30
wavelength=c/(carrier_frequency*(10**9)) #Becuase freq in ghz
number_of_flows=80
number_of_time_slots=2000
slot_duration=0.1
fading_depth=2
background_noise=-134
system_bandwidth=2000
transmission_power=40
average_power_of_uav=30
peak_power_of_uav=2*average_power_of_uav
transceiver_efficiency=0.8
height_of_uav=100
speed_of_uav=20
pl_factor_for_v2v=2.5
pl_factor_for_u2v=2
rician_k_factor=9
interference_threshold=1/(10**3)
si_cancellation_level=1/(10**9)
maximum_antenna_gain=20
half_power_beamwidth=30
total_power = 1
number_of_uavs = 10
distance_u2v=np.zeros((number_of_uavs+1, number_of_flows+1))
distance_v2u=np.zeros((number_of_flows+1, number_of_uavs+1))
small_scale_power_fading_matrix_u2v = np.zeros((number_of_uavs+1, number_of_flows+1))
small_scale_power_fading_matrix_v2u = np.zeros((number_of_flows+1, number_of_uavs+1))
channel_power_gain_matrix=np.zeros((number_of_flows+1, number_of_flows+1))
received_signal_power_matrix=np.zeros((number_of_flows+1, number_of_flows+1))
distance_matrix=np.zeros((number_of_flows+1, number_of_flows+1))
# 2-D Binary Matrices
a_v2v=np.zeros((number_of_flows+1, number_of_flows+1))
b_v2v=np.zeros((number_of_flows+1, number_of_flows+1))
a_u2v=np.zeros((number_of_flows+1, number_of_flows+1))
b_u2v=np.zeros((number_of_flows+1, number_of_flows+1))

In [4]:
# Conversion functions to standardize constants

def dbm_to_watts(dbm):
    """Converts dBm (decibels relative to one milliwatt) to Watts."""
    return 10**(dbm / 10) / 1000

def watts_to_dbm(watts):
    """Converts Watts to dBm (decibels relative to one milliwatt)."""
    return 10 * math.log10(1000 * watts)

def dBi_to_linear_scale(dBi):
    """Converts dBi (decibels relative to an isotropic radiator) to linear scale."""
    return 10**(dBi / 10)

def linear_scale_to_dBi(linear_scale):
    """Converts linear scale to dBi (decibels relative to an isotropic radiator)."""
    return 10 * math.log10(linear_scale)



In [5]:
def section_formula(x1, y1, x2, y2, m, n):
    Px = (m * x2 + n * x1) / (m + n)
    Py = (m * y2 + n * y1) / (m + n)
    return Px, Py

In [6]:
def calculate_antenna_gain(direction: float): #To be implemented
    deduction=min((direction/half_power_beamwidth)**2,26)
    gain=maximum_antenna_gain-deduction
    return maximum_antenna_gain

In [7]:
# g here is channel power gain and m is parameter for Gama pehelwan function
def calculate_channel_power_gain_distribution():
    for i in range(1,number_of_flows+1):
        for j in range(1,number_of_flows+1):
            channel_power_gain_matrix[i, j]=np.random.gamma(shape=fading_depth, scale=1/fading_depth)

In [8]:
def calculate_received_signal_power_direct_link(i):
    kv=(wavelength/(4*math.pi))**pl_exponent
    signal_power=kv*dbm_to_watts(transmission_power)*dBi_to_linear_scale(maximum_antenna_gain)*channel_power_gain_matrix[i, i]*(distance_matrix[i, i]**(-pl_exponent))
    return signal_power

In [9]:
def calculate_mutual_interference(j, i):
    kv=(wavelength/(4*math.pi))**pl_exponent
    signal_power=kv*dbm_to_watts(transmission_power)*dBi_to_linear_scale(calculate_antenna_gain(0))*channel_power_gain_matrix[j, i]*(distance_matrix[j, i]**(-pl_exponent))
    return signal_power

In [10]:
def calculate_received_signal_power():
    for i in range(number_of_flows):
        for j in range(number_of_flows):
            if i==j:
                received_signal_power_matrix[i, i]=calculate_received_signal_power_direct_link(i)
            else:
                received_signal_power_matrix[j, i]=calculate_mutual_interference(j, i)  

In [11]:
# beta= cancellation level
def calculate_rsi():
    return si_cancellation_level*dbm_to_watts(transmission_power)

In [12]:
# n=no of flows, n0 = noise power spectral density, w = channel bandwidth
# eta= efficiency
def calculate_sinr_and_datarate_v2v():
    rsi = calculate_rsi()
    sinr=np.empty(number_of_flows+1)
    datarate=np.empty(number_of_flows+1)
    for i in range(1,number_of_flows+1):
        num=received_signal_power_matrix[i, i]
        denom= dbm_to_watts(background_noise*system_bandwidth)
        for j in range(1,number_of_flows+1):
            if j==i:
                continue
            denom+=(a_v2v[j, i]*received_signal_power_matrix[j, i])+(b_v2v[j, i]*rsi)
        sinr[i]=num/denom
        datarate[i]=transceiver_efficiency*system_bandwidth*math.log2(1+sinr[i])
    return sinr,datarate

In [13]:
def calculate_non_central_chi_square_distribution(small_scale_power_fading, total_power):
    ans=(rician_k_factor + 1)*np.exp(-rician_k_factor)*np.exp(-(rician_k_factor+1)*small_scale_power_fading/total_power)
    ans*=iv(0, 2*np.sqrt(rician_k_factor*(rician_k_factor+1)*small_scale_power_fading/total_power))/total_power
    return ans

In [14]:
def calculate_small_scale_power_fading():
    for i in range(1,number_of_uavs+1):
        for j in range(1,number_of_flows+1):
            small_scale_power_fading = np.random.uniform(0, np.inf)
            small_scale_power_fading_matrix_u2v[i, j] = calculate_non_central_chi_square_distribution(small_scale_power_fading, total_power)
            
            small_scale_power_fading = np.random.uniform(0, np.inf)
            small_scale_power_fading_matrix_v2u[j, i] = calculate_non_central_chi_square_distribution(small_scale_power_fading, total_power)

In [15]:
def calculate_sinr_and_datarate_u2v(transmit_power_uav=30):
    rsi = calculate_rsi()
    sinr=np.empty((number_of_uavs+1,number_of_flows+1))
    datarate=np.empty((number_of_uavs+1, number_of_flows+1))
    ku=(wavelength/(4*math.pi))**pl_factor_for_u2v
    
    for u in range(1, number_of_uavs+1):
        for k in range(1,number_of_flows+1):
            num=ku*dbm_to_watts(transmit_power_uav)*dBi_to_linear_scale(maximum_antenna_gain)*(distance_u2v[u, k]**(-pl_factor_for_u2v))*small_scale_power_fading_matrix_u2v[u, k]
            denom= dbm_to_watts(background_noise*system_bandwidth)
            for w in range(1,number_of_flows+1):
                if w==k:
                    continue
                denom+=(a_u2v[w, k]*received_signal_power_matrix[w, k])+(b_u2v[w, k]*rsi)
            sinr[u, k]=num/denom
            # sinr[u, k]=denom
            datarate[u, k]=transceiver_efficiency*system_bandwidth*math.log2(1+sinr[u, k])
        
    return sinr,datarate

In [16]:
def calculate_sinr_and_datarate_v2u():
    sinr=np.empty((number_of_flows+1, number_of_uavs))
    datarate=np.empty((number_of_flows+1, number_of_uavs))
    kv=(wavelength/(4*math.pi))**pl_factor_for_v2v
    
    for k in range(1,number_of_flows+1):
        for u in range(1,number_of_uavs+1):
            num=kv*dbm_to_watts(transmission_power)*dBi_to_linear_scale(maximum_antenna_gain)*(distance_v2u[u, k]**(-pl_factor_for_v2v))*small_scale_power_fading_matrix_v2u[k, u]
            denom=dbm_to_watts(background_noise*system_bandwidth)
            sinr[k, u]=num/denom
            datarate[k, u]=transceiver_efficiency*system_bandwidth*math.log2(1+sinr[k, u])
    
    return sinr,datarate

# RI (Relative Interference )

1. ek ka sender dusre ka receiver hai and ek ka receiver dusre ka sender hai
2. ek ka sender dusre ka receiver hai or ek ka receiver dusre ka sender hai
3. humpe hai hi 9 (no clash sword fight cancelled)


In [17]:
def calculate_relative_intereference1():
    num = dbm_to_watts(background_noise*system_bandwidth) + calculate_rsi()
    denom = transmission_power
    return num / denom

In [18]:
def calculate_relative_intereference2(sender,receiver):
    num = dbm_to_watts(background_noise*system_bandwidth) + received_signal_power_matrix[receiver,sender]
    denom = transmission_power
    return num / denom